In [1]:
from opensearchpy import helpers, OpenSearch

In [4]:
stg_vpce_opensearch = "aos-ef399bbbead7-qs5jilysrng2oxiiayoxwtklki.ap-northeast-2.es.amazonaws.com"
PORT = 443
index_name = 'onemodel_v3_inital_test_20240512'

In [5]:
index_doc = {"index": index_name}

In [6]:
client = OpenSearch(
    hosts = [{"host": stg_vpce_opensearch, "port": PORT}],
    http_compress=True,
    use_ssl=True,
    verify_certs=True,
    timeout=2,
    pool_maxsize=40,
    http_auth=("TheMasterUser", "Developer_rec_stg1!"),
)

In [7]:
from typing import List

In [34]:
w = {}
w.get('size', 5)

5

In [64]:
def get_base_query(source:List=[], must_query:List=[], should_query:List=[], filter_query:dict={}, W:dict={}, function_score_query:dict={}, functions:List=[]):

    must_not_filter = filter_query.get('must_not', [])
    must_filter = filter_query.get('must', [])
    size = W.get("size", 5)

    base_query_dsl = {
        "_source":source,
        "size": size,
        "query": {
            "bool": {
                        "must": [{
                                    "bool":{
                                        "should": must_query,
                                        "minimum_should_match": 1
                                    }
                                }],
                        "should": should_query,
                        "filter":[{
                                "bool":{
                                    "must_not": must_not_filter,
                                    "must": must_filter 
                                    }
                        }]
                    }
            },
        "sort" : ["_score"],
    }
    return base_query_dsl


In [121]:
must_query = [{"multi_match":{"query": "게임 좋아하는 사람", "fields": ['mno_profiles', 'adot_profiles']}}]
source =["svc_mgmt_num", "mno_profiles", "adot_profiles"]

In [122]:
get_query = get_base_query(source=source, must_query=must_query)

In [123]:
response = client.search(
    body = get_query,
    index = index_name
)

In [124]:
response

{'took': 57,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 65, 'relation': 'eq'},
  'max_score': 8.8994665,
  'hits': [{'_index': 'onemodel_v3_inital_test_20240512',
    '_id': 'e258ef6adbc20b4f76262cb45eb7f19a15e9bcee9329eec736372aec9014f477',
    '_score': 8.8994665,
    '_source': {'svc_mgmt_num': 'e258ef6adbc20b4f76262cb45eb7f19a15e9bcee9329eec736372aec9014f477',
     'mno_profiles': '관심사:음식/식사_카페,미디어/엔터테인먼트_음악,차량/운송_차 구매<|n|>성별:여자<|n|>나이:35세<|n|>서비스 사용 기간:8년<|n|>기변 후 경과일:1278일<|n|>요금제 이름:T플랜 스페셜<|n|>요금제 가격:79000원<|n|>3개월 평균 데이터 사용량:85gb<|n|>단말기 가격:고가<|n|>단말기 제조사:apple<|n|>멤버십 등급:vip<|n|>멤버십 사용 이력:없음<|n|>가족 결합 이력:있음<|n|>로밍 사용 이력:없음<|n|>세컨디바이스 보유 여부:없음<|n|>소액 및 DCB 결제 이력:0원',
     'adot_profiles': '선호 도메인 : 게임<|n|>선호 카테고리 : 게임(하이퍼캐쥬얼) <|n|>선호 아이템 : 게임(프리셀, 틀린그림찾기) <|n|>성별 : 여성<|n|>나이 : 35<|n|>활성 상태 : 활성<|n|>재방문 비율 : 없음<|n|>다중 도메인 성향 : 보통<|n|>인기 컨텐츠 선호도 : 낮음<|n|>사용성 기준 : 일반유저<|n|>헤비유저인 도메인 : 루틴, T 서비스, 게임'}},
 